# [TF] Pawpularity EfficientNetB0 + metadata ensamble 😺 🐶

![](https://www.petfinder.my/images/cuteness_meter.jpg)

# Description of the notebook:

- The architecture aims to model images and tabular data
- The base model used for images is EfficientNet b0 with noisy student weights
- Trained with AdamW + cosine decay 
- Performed an ensamble using 5-fold CV

In [ ]:
lazy_submit = False # set to False to train, set False to use the stored trained weights
COLAB = False

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.random import set_seed
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import os
import gc
import cv2
from tqdm import tqdm
import seaborn as sns
import tensorflow_addons as tfa
import math
import xgboost as xgb
import optuna
from optuna.integration import XGBoostPruningCallback
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import json
import pickle

In [ ]:
print(tf. __version__)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

In [ ]:
SEED = 42
np.random.seed(SEED)
set_seed(SEED)

IMAGE_SIZE = (224, 224)

BATCH_SIZE = 32
EPOCHS = 10
FOLDS = 5
MULTIVARIATE = True

if COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')
  datapath = "gdrive/MyDrive/PAWPULARITY"
else:
  datapath  = ".."

INPUT_DIR = os.path.join(datapath, "input")
OUTPUT_DIR = os.path.join(datapath, "working")
TRAIN_IMAGES_DIR = os.path.join(INPUT_DIR, "petfinder-pawpularity-score", 'train')
TRAIN_DS = os.path.join(INPUT_DIR, "petfinder-pawpularity-score", 'train.csv')
TEST_IMAGES_DIR = os.path.join(INPUT_DIR, "petfinder-pawpularity-score", 'test')
TEST_DS = os.path.join(INPUT_DIR, "petfinder-pawpularity-score", 'test.csv')
SUBMISSION_DS = os.path.join(INPUT_DIR, "petfinder-pawpularity-score", 'sample_submission.csv')

if lazy_submit:
    TRAINED_WEIGHTS_DIR = os.path.join(INPUT_DIR,  'weights-pawpularity')
else: 
    TRAINED_WEIGHTS_DIR = OUTPUT_DIR
    
weights =  os.path.join(INPUT_DIR, "efficientnet-noisy-student-b0", "efficientnetb0_noisy_student_notop.h5")

In [ ]:
!nvidia-smi

In [ ]:
train_ds = pd.read_csv(TRAIN_DS)
train_ds['Pawpularity'] = train_ds['Pawpularity']/100.0
test_ds = pd.read_csv(TEST_DS)
subm_ds = pd.read_csv(SUBMISSION_DS)
meta_cols = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 
             'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
train_ds.shape

# Exploratory analysis

In [ ]:
grouped = train_ds.copy()
grouped["cuts"] = pd.cut(train_ds["Pawpularity"], bins = np.arange(0, 1.1, 0.1),labels= np.round(np.arange(0, 1, 0.1), 1))
grouped = grouped.drop("Id", axis = 1).groupby("cuts").mean()
fig,ax = plt.subplots(6, 2, sharex = False, sharey = True, figsize = (14, 16))
col = 0
for i in range(6):
    for j in range(2):
        sns.barplot(ax = ax[i, j], x = meta_cols[col], y = "cuts",  data = grouped[meta_cols].reset_index())
        col += 1
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (10, 8))
sns.heatmap(grouped.corr())

In [ ]:
plt.figure(figsize = (10, 8))
sns.clustermap(grouped.corr())

In [ ]:
img = keras.preprocessing.image.load_img("../input/petfinder-pawpularity-score/train/00524dbf2637a80cbc80f70d3ff59616.jpg")
img = keras.preprocessing.image.img_to_array(img) / 255.0
img.shape

In [ ]:
plt.imshow(img)

## Useful functions

In [ ]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df, img_dir, 
                 batch_size, tab_columns,
                 id, target, is_train,
                 input_size = (224, 224),
                 shuffle = True):
        
        self.df = df.copy()
        self.img_dir = img_dir
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle    
        self.n = len(self.df)
        self.tab_columns = tab_columns
        self.target = target
        self.id = id
        self.is_train = is_train
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.ceil(len(self.df) / float(self.batch_size)))
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

    def __getitem__(self, index):
        this_ds = self.df.iloc[index*self.batch_size:(index+1)*self.batch_size]
        images = []
        for img_id in list(this_ds[self.id].values):
            img = cv2.imread(f"{self.img_dir}/{img_id}.jpg")
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, IMAGE_SIZE, interpolation=cv2.INTER_LINEAR)
            img = np.array(img, dtype='float32') 
            # ./ 255.0 EfficientNet models expect their inputs to be float tensors of pixels with values in the [0-255] range
            images.append(img)
        if self.is_train:
            return [np.array(images), this_ds[self.tab_columns].values],this_ds[self.target].values
        else:
            return [np.array(images), this_ds[self.tab_columns].values]
        

def get_intermediate_layer_output(model, x_train):
    intermediate_layer_model = keras.Model(inputs=model.input,
                                 outputs=model.layers[-3].output)
    intermediate_output = intermediate_layer_model.predict(x_train, verbose = 1) 
    return pd.DataFrame(data=intermediate_output)


def run(trial, xtrain, xval, ytrain, yval):
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 12)
    
    pruning_callback = XGBoostPruningCallback(trial, "validation_0-rmse")

    model = XGBRegressor(
        objective ="reg:logistic",
        random_state = SEED,
        tree_method="gpu_hist",
        n_estimators= 3000,
        predictor="gpu_predictor",
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth)

    model.fit(xtrain,ytrain, 
             early_stopping_rounds= 100, 
             eval_set= [(xval, yval)],
             callbacks = [pruning_callback],
             eval_metric='rmse', verbose = True)
             
    preds_valid = model.predict(xval)
    rmse = mean_squared_error(yval, preds_valid, squared=False)
    return rmse

## Build the model

In [ ]:
NCOL = len(meta_cols)
INPUT_SHAPE = (*IMAGE_SIZE, 3)


class CosineAnnealingStairCase(tf.keras.callbacks.Callback):
    def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
        self.epochs = n_epochs
        self.cycles = n_cycles
        self.lr_max = lrate_max
        self.lrates = list()
        self.best = 1000
        self.log = -1000
        self.rate = 1

    def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max, rate):
        epochs_per_cycle = math.floor(n_epochs/n_cycles)
        cos_inner = (math.pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle) 
        return lrate_max/2 * (math.cos(cos_inner) + 1) / rate

    def on_epoch_begin(self, epoch, logs=None):
        if self.log > self.best:
            self.rate  *= 1.5
        lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max, self.rate)
        print(f'Learning rate: {lr}')
        tf.keras.backend.set_value(self.model.optimizer.lr, lr)
        
    def on_epoch_end(self, epoch, logs = {}):
        self.log = logs['val_root_mean_squared_error'] 
        if self.log < self.best:
            self.best = self.log


def create_model():

    CONV_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 2.0,
        'mode': 'fan_out',
        'distribution': 'normal'
        }
    }

    conv_base = tf.keras.applications.efficientnet.EfficientNetB0(
        weights = weights,
        include_top = False,
        input_tensor=keras.Input(shape = INPUT_SHAPE))

    conv_base.trainable = False

    data_augmentation = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.RandomContrast(0.2, seed = SEED),
            tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", seed = SEED),
            tf.keras.layers.experimental.preprocessing.RandomRotation(0.2, seed = SEED),
            tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor = 0.1, width_factor = 0.1, seed = SEED)
            #tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=(0.2, 0.3), width_factor=(0.2, 0.3),  seed=SEED),
        ])
    
    def conv2d_block(mod, filters):
        out = layers.Conv2D(filters, kernel_size  = (1, 1),
                          padding='same',
                          kernel_initializer = CONV_KERNEL_INITIALIZER)(mod)
        out = layers.BatchNormalization()(out)
        out = layers.Activation("swish")(out)
        return out

    inp = tf.keras.Input(shape=INPUT_SHAPE)
    out = data_augmentation(inp)
    out = conv_base(out)
    out = conv2d_block(out, 640)
    out = conv2d_block(out, 320)
    out = conv2d_block(out, 128)
    out = layers.GlobalAveragePooling2D()(out)
    out = layers.BatchNormalization()(out)
    out = layers.Dropout(0.1)(out) 

    meta_input = keras.Input(shape = (NCOL,))
    concat = layers.Concatenate(axis = 1)([out, meta_input])
    concat = layers.Dense(64)(concat) 
    concat = layers.PReLU()(concat)
    concat = layers.Dropout(0.1)(concat) 
    concat = layers.Dense(1, activation = "sigmoid")(concat)
    model = tf.keras.Model([inp, meta_input], concat)

    return model

In [ ]:
#check_model = create_model()
#check_model.compile()
#check_model.summary()

In [ ]:
#tf.keras.utils.plot_model(check_model, show_shapes=True)

## Train the model

In [ ]:
kf = KFold(n_splits = FOLDS, shuffle = True, random_state = SEED)

def train():
   
    nn_models = []
    xgb_models = []
    histories = []
    rmse_out = 0
    for idx, (train_idx, val_idx) in enumerate(kf.split(train_ds)):
        
        print(f'\n----- RUN: {idx} ------')
        tf.keras.backend.clear_session()
        gc.collect()
        
        params = dict(batch_size=32, tab_columns=meta_cols, id="Id", target="Pawpularity", is_train = True)
        train_gen = CustomDataGen(img_dir = TRAIN_IMAGES_DIR, df=train_ds.copy().iloc[train_idx], **params)
        val_gen = CustomDataGen(img_dir = TRAIN_IMAGES_DIR, df=train_ds.copy().iloc[val_idx], **params)
        
        optimizer = tfa.optimizers.AdamW(lr=1e-3, weight_decay=1e-4)

        es = tf.keras.callbacks.EarlyStopping(
            monitor='val_root_mean_squared_error', patience=3)
        
        ca = CosineAnnealingStairCase(EPOCHS, 1, 1e-3)
        
        checkpoint = tf.keras.callbacks.ModelCheckpoint(os.path.join(OUTPUT_DIR, f'efficientnetb0_{idx}.h5'), 
                                                    monitor='val_root_mean_squared_error', verbose = 1, 
                                                    save_best_only=True, mode='min', save_weights_only = True)
        
        rmse = tf.keras.metrics.RootMeanSquaredError()

        model = create_model()
        model.compile(loss = "binary_crossentropy", 
            optimizer = optimizer,
            metrics=[rmse])

        history = model.fit(train_gen,
                            validation_data = val_gen,
                            epochs = EPOCHS, 
                            callbacks = [checkpoint, es, ca],
                            max_queue_size = 3 * BATCH_SIZE)
               
        nn_models.append(model)
        histories.append(history.history)
        
        params = dict(batch_size=32, tab_columns=meta_cols, id="Id", target="Pawpularity", is_train = True, shuffle = False)
        train_gen = CustomDataGen(img_dir = TRAIN_IMAGES_DIR, df=train_ds.iloc[train_idx], **params)
        val_gen = CustomDataGen(img_dir = TRAIN_IMAGES_DIR, df=train_ds.iloc[val_idx], **params)
        x_train = get_intermediate_layer_output(nn_models[idx], train_gen)
        x_val = get_intermediate_layer_output(nn_models[idx], val_gen)
        
        sampler = TPESampler(seed=SEED, multivariate=MULTIVARIATE)
        study = optuna.create_study(direction="minimize", sampler = sampler)
        study.optimize(lambda trial: run(trial, x_train, x_val, train_ds.iloc[train_idx].loc[:, 'Pawpularity'], train_ds.iloc[val_idx].loc[:, 'Pawpularity']), n_trials=100)
        best = study.best_params
        with open(os.path.join(OUTPUT_DIR, f'best_{idx}.json'), 'w') as f:
            json.dump(best, f)
        print(best)
        
        model = XGBRegressor(
        objective="reg:logistic",
        random_state=42,
        n_estimators= 5000,
        **best)

        model.fit(x_train,  train_ds.iloc[train_idx].loc[:, 'Pawpularity'], 
                    early_stopping_rounds= 100, 
                    eval_set= [(x_val, train_ds.iloc[val_idx].loc[:, 'Pawpularity'])],
                    eval_metric='rmse', verbose = True)

        preds_valid = model.predict(x_val)
        rmse = mean_squared_error(train_ds.iloc[val_idx].loc[:, 'Pawpularity'], preds_valid, squared=False)
        rmse_out += rmse
        print('RMSE: ', rmse)

        with open(os.path.join(OUTPUT_DIR, f'model_xgb_{idx}.pkl'), 'wb') as f:
            pickle.dump(model, f)
        xgb_models.append(model)
    
        gc.collect()
        
    return nn_models,xgb_models, histories, rmse_out / FOLDS

In [ ]:
if not lazy_submit:
    nn_models,xgb_models, histories, rmse  = train()
    evals = np.mean([np.min(histories[i]['val_root_mean_squared_error']) for i in range(FOLDS)])
    
    fig,ax = plt.subplots(3,2)
    for i in range(FOLDS):
        ax = ax.flatten()
        ax[i].plot(histories[i]['loss'], label= 'loss')
        ax[i].plot(histories[i]['val_loss'], label = 'val_loss')
        ax[i].legend()
    plt.show()
    
    evals
    
else:
    models = []
    evals = 0
    for idx, (train_idx, val_idx) in enumerate(kf.split(train_ds)):
        params = dict(img_dir=TRAIN_IMAGES_DIR, batch_size=BATCH_SIZE, tab_columns=meta_cols, id="Id", target="Pawpularity", is_train = True, shuffle  = False)
        val_gen = CustomDataGen(df=train_ds.iloc[val_idx], **params)
        model = get_model()
        model.load_weights(os.path.join(TRAINED_WEIGHTS_DIR, f'efficientnetb0_{idx}.h5'))
        models.append(model)
        evals += model.evaluate(val_gen, 
                                    max_queue_size = 3 * BATCH_SIZE)[1] 
    evals /= FOLDS

print("AVG RMSE NN: ", evals)
print("AVG RMSE XGB: ", rmse)

# Submission

In [ ]:
predictions = []
params = dict(img_dir = TEST_IMAGES_DIR, batch_size = BATCH_SIZE, tab_columns = meta_cols, id = "Id", target = "Pawpularity", is_train = False, shuffle = False)
test_gen = CustomDataGen(df = test_ds, **params)

for i in tqdm(range(FOLDS)):
    x_val = get_intermediate_layer_output(nn_models[i], test_gen)
    predictions.append(xgb_models[i].predict(test_gen))
    
subm_ds["Pawpularity"] =  100 * np.array(predictions).mean(axis = 0)
subm_ds.to_csv("submission.csv", index = False)

subm_ds.head()